<a href="https://colab.research.google.com/github/AntoninaKOR/training-code-ml/blob/main/bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Данный блокнот был сделан с целью познакомиться с библиотекой transformers. Спасибо статье за упрощение этой возможности https://medium.com/m/global-identity-2?redirectUrl=https%3A%2F%2Ftowardsdatascience.com%2Fmake-your-own-rick-sanchez-bot-with-transformers-and-dialogpt-fine-tuning-f85e6d1f4e30

In [ ]:
! pip install transformers==4.22.2
! pip install tensorboardX
! pip install apex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 104.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.8/709.8 kB 33.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.3/247.3 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.2/145.2 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.0/115.0 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

# Подгрузка библиотек

In [ ]:
import pandas as pd
import numpy as np
import torch
from typing import Dict, List, Tuple
import glob
import os
import random
import re
import shutil

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split

try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

# Пробуем бота без fune-tuning


In [ ]:

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small", padding_size="left")
model = AutoModelWithLMHead.from_pretrained("microsoft/DialoGPT-small")

(…)small/resolve/main/tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

(…)/DialoGPT-small/resolve/main/config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

(…)t/DialoGPT-small/resolve/main/vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

(…)t/DialoGPT-small/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1062: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/351M [00:00<?, ?B/s]

In [ ]:
for step in range(3):
    # кодируем наш ввод
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    chat_history_ids = model.generate(
    bot_input_ids, max_length=1000,
    pad_token_id=tokenizer.eos_token_id
    )
    print("GPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User:Hi
GPT: Hi
>> User:Are you a student?
GPT: I'm a student and I can confirm this.
>> User:what is your name?
GPT: I'm a student and I can confirm this.


# Обрабатываем данные

In [ ]:
class Args():
    def __init__(self):
        self.output_dir = 'output-small'
        self.model_type = 'gpt2'
        self.model_name_or_path = 'microsoft/DialoGPT-small'
        self.config_name = 'microsoft/DialoGPT-small'
        self.tokenizer_name = 'microsoft/DialoGPT-small'
        self.cache_dir = 'cached'
        self.block_size = 512
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 4
        self.per_gpu_eval_batch_size = 4
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 3
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 3500
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.should_continue = False
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False
        self.fp16_opt_level = 'O1'

args = Args()

In [ ]:
df= pd.read_csv("chizuru_ichinose6.csv")
df

,name,line
0,Chizuru Ichinose,An orange shirt and striped pants…
1,Chizuru Ichinose,"You must be Kazuya-Kun, right?"
2,Chizuru Ichinose,You’re in college? What year?
3,Kazuya Kinoshita,F-First year.
4,Chizuru Ichinose,"What? Me, too!"
...,...,...
2231,Chizuru Ichinose,She’s not likely to leave until you give her t...
2232,Kazuya Kinoshita,"W-Well, yeah…"
2233,Chizuru Ichinose,She even knows our college.
2234,Chizuru Ichinose,It’d be too risky to leave her to her own devi...


In [ ]:
df['name'].unique()

array(['Chizuru Ichinose', 'Kazuya Kinoshita', 'Cashier',
       'Chizuru Ichinose ', 'Grandma Nagomi', 's Mother', 's Father',
       'san', 'Grandma Sayuri', 'kun', 'Kun', 'Kazuya Kinoshita ', 'chan',
       'Random guy', 'chan ', ' Chizuru Ichinose', 'Waitress',
       'Random grandma'], dtype=object)

У персонажей из субтитров перед/после именами могу стоять пробелы из субтитров. Нам достаточно привести к единой форме только имя Тидзуру (Chizuru Ichinose).

In [ ]:
df.loc[(df['name'] == ' Chizuru Ichinose') | (df['name'] == 'Chizuru Ichinose '), 'name'] = 'Chizuru Ichinose'
n = df['name'].unique()
n

array(['Chizuru Ichinose', 'Kazuya Kinoshita', 'Cashier',
       'Grandma Nagomi', 's Mother', 's Father', 'san', 'Grandma Sayuri',
       'kun', 'Kun', 'Kazuya Kinoshita ', 'chan', 'Random guy', 'chan ',
       'Waitress', 'Random grandma'], dtype=object)

Теперь пересобираем датасет. Он будет иметь вид фраза от Chizuru Ichinose, затем 6 строк диалога до этого ответа, и так с каждой ее фразой.

In [ ]:
df_ans_q= df[df['name']=='Chizuru Ichinose']
l=[]
for i in df_ans_q.index[4::]:
  l.extend(range(i-6,i+1))

In [ ]:
df = df.iloc[l]
df

,name,line
0,Chizuru Ichinose,An orange shirt and striped pants…
1,Chizuru Ichinose,"You must be Kazuya-Kun, right?"
2,Chizuru Ichinose,You’re in college? What year?
3,Kazuya Kinoshita,F-First year.
4,Chizuru Ichinose,"What? Me, too!"
...,...,...
2231,Chizuru Ichinose,She’s not likely to leave until you give her t...
2232,Kazuya Kinoshita,"W-Well, yeah…"
2233,Chizuru Ichinose,She even knows our college.
2234,Chizuru Ichinose,It’d be too risky to leave her to her own devi...


In [ ]:
df.reset_index(drop= True , inplace= True )

Теперь делаем итоговый датасет в котором в первом столбце фраза от Chizuru Ichinose, затем 6 фраз до в 6 других колонках.

In [ ]:
contexted = []

n = 6

for i in range(n, len(df['line'])):
  row = []
  prev = i - 1 - n
  for j in range(i, prev, -1):
    row.append(df['line'][j])
  contexted.append(row)

In [ ]:
columns = ['response', 'context']
columns = columns + ['context/'+str(i) for i in range(n-1)]
columns

['response',
 'context',
 'context/0',
 'context/1',
 'context/2',
 'context/3',
 'context/4']

In [ ]:
df = pd.DataFrame.from_records(contexted, columns=columns)
df.head(5)

,response,context,context/0,context/1,context/2,context/3,context/4
0,"Ah, you are older than me then.","Oh, but I took a gap year.","What? Me, too!",F-First year.,You’re in college? What year?,"You must be Kazuya-Kun, right?",An orange shirt and striped pants…
1,"You must be Kazuya-Kun, right?","Ah, you are older than me then.","Oh, but I took a gap year.","What? Me, too!",F-First year.,You’re in college? What year?,"You must be Kazuya-Kun, right?"
2,You’re in college? What year?,"You must be Kazuya-Kun, right?","Ah, you are older than me then.","Oh, but I took a gap year.","What? Me, too!",F-First year.,You’re in college? What year?
3,F-First year.,You’re in college? What year?,"You must be Kazuya-Kun, right?","Ah, you are older than me then.","Oh, but I took a gap year.","What? Me, too!",F-First year.
4,"What? Me, too!",F-First year.,You’re in college? What year?,"You must be Kazuya-Kun, right?","Ah, you are older than me then.","Oh, but I took a gap year.","What? Me, too!"


In [ ]:
trn_df, val_df = train_test_split(df, test_size = 0.1)

Теперь в класс ConversationDataset помещаем строки датасета, пропущенные через токенизатора.

In [ ]:
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

class ConversationDataset(Dataset):
  def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):

    block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)
    self.examples = []
    for _, row in df.iterrows():
      conv = construct_conv(row, tokenizer)
      self.examples.append(conv)

  #я убрала часть кода с сохранением обработанных токенизатором строк в файле, и подгрузка их после сохранения, так как выбранный датасет достаточно маленький.

  def __len__(self):
    return len(self.examples)

  def __getitem__(self, item):
    return torch.tensor(self.examples[item], dtype=torch.long)

# Работа с версиями

In [ ]:
def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
    return ConversationDataset(tokenizer, args, df_val if evaluate else df_trn)

#фиксируем для воспроизводимости результата.
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

#функция возвращает лист сохраненных чекпоинтев.
def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []

    glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))

    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args.save_total_limit:
        return
    if args.save_total_limit <= 0:
        return

    # удаляем лишнии чекпоинты
    checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args.save_total_limit:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        print("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
        shutil.rmtree(checkpoint)

# Обучение

In [ ]:
def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)
    #выбираем sampler в зависимости от того можем мы обучать на нескольких gpu или нет
    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )
    #Нам необходимо пересчитать общее количество шагов обучения, поскольку размер загрузчика обучающих данных мог измениться
    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # параллельное обучение
    model.resize_token_embeddings(len(tokenizer))
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # проверяем есть ли сохраненный optimizer или scheduler
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi gpu
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # распледеленное обучение
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # обучение
    print("***** Running training *****")
    print("  Num examples = %d", len(train_dataset))
    print("  Num Epochs = %d", args.num_train_epochs)
    print("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    print(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    print("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    print("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # проверка продолжаем ли обучение
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # устанавливаем global_step рвный gobal_step сохраненниго checkpoint
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps) #num_update_steps_per_epoch = len(train_dataloader) // args.gradient_accumulation_steps
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            print("  Continuing training from checkpoint, will skip to saved global_step")
            print("  Continuing training from epoch %d", epochs_trained)
            print("  Continuing training from global step %d", global_step)
            print("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            print("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # для воспроизводимости
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):

            # пропускаем уже сделанные шаги
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]

            if args.n_gpu > 1:
                loss = loss.mean()  # усреднение
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:

                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # сохраняем checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # параллельное обучение
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    print("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    print("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

# Evaluation (оценивание)

def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # DistributedSampler семплирует случайно

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # оценка
    print("***** Running evaluation {} *****".format(prefix))
    print("  Num examples = %d", len(eval_dataset))
    print("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        print("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            print("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

Теперь используем все написанные до этого функции.

In [ ]:
args = Args()
#загружаем чекпоинты.
if args.should_continue:
  sorted_checkpoints = _sorted_checkpoints(args)
  if len(sorted_checkpoints) == 0:
    raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
  else:
    #берем самый последний чекпоинт.
    args.model_name_or_path = sorted_checkpoints[-1]
if (
  os.path.exists(args.output_dir)
  and os.listdir(args.output_dir)
  and args.do_train
  and not args.overwrite_output_dir
  and not args.should_continue
    ):
  raise ValueError("Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir))

device = torch.device("cuda")
args.n_gpu = torch.cuda.device_count()
args.device = device
#показываем параметры, на которых будет обучаться модель.
print(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

set_seed(args)

config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
model = AutoModelWithLMHead.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
        cache_dir=args.cache_dir,
)
model.to(args.device)
print("Training/evaluation parameters %s", args)

# обучение
if args.do_train:
  train_dataset = load_and_cache_examples(args, tokenizer, trn_df, val_df, evaluate=False)
  global_step, tr_loss = train(args, train_dataset, model, tokenizer)
  print(" global_step = %s, average loss = %s", global_step, tr_loss)
if args.do_train:
  os.makedirs(args.output_dir, exist_ok=True)
  print("Saving model checkpoint to %s", args.output_dir)
  model_to_save = (model.module if hasattr(model, "module") else model)
  model_to_save.save_pretrained(args.output_dir)
  tokenizer.save_pretrained(args.output_dir)
  torch.save(args, os.path.join(args.output_dir, "training_args.bin"))
  model = AutoModelWithLMHead.from_pretrained(args.output_dir)
  tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
  model.to(args.device)

# Оценивание
results = {}
if args.do_eval and args.local_rank in [-1, 0]:
  checkpoints = [args.output_dir]
  if args.eval_all_checkpoints:
    checkpoints = list(os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True)))
  print("Evaluate the following checkpoints: %s", checkpoints)
  for checkpoint in checkpoints:
    global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
    prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

    model = AutoModelWithLMHead.from_pretrained(checkpoint)
    model.to(args.device)
    result = evaluate(args, model, tokenizer, trn_df, val_df, prefix=prefix)
    result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
    results.update(result)

Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s -1 cuda 1 False False


(…)/DialoGPT-small/resolve/main/config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

(…)small/resolve/main/tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

(…)t/DialoGPT-small/resolve/main/vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

(…)t/DialoGPT-small/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:265: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Iteration:   0%|          | 0/823 [00:00<?, ?it/s]

 global_step = %s, average loss = %s 2469 0.8598133741348752
Saving model checkpoint to %s output-small
Evaluate the following checkpoints: %s ['output-small']
***** Running evaluation  *****
  Num examples = %d 367
  Batch size = %d 4


Evaluating:   0%|          | 0/91 [00:00<?, ?it/s]

***** Eval results  *****
  %s = %s perplexity tensor(1.4153)


# Модель после fune-tuning

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
model = AutoModelWithLMHead.from_pretrained('output-small')

for step in range(5):
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
    chat_history_ids = model.generate(
        bot_input_ids, max_length=200,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=100,
        top_p=0.7,
        temperature = 0.8
    )
    print("ChizuruBot: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User:Hi
ChizuruBot: Hey! 
>> User:Are you student?
ChizuruBot: I’m glad it wasn’t anything major.
>> User:What is your name?
ChizuruBot: Kazuya-san? 
>> User:Kazuya?
ChizuruBot: !!!?!!
>> User:What?
ChizuruBot: ,!!,!?


# Выводы

Модель речь стала больше похожа на персонажа (модель запомнила некоторую информацию из самого аниме, например, имя Казуя, но бот все также отвечает насвязанно.
Скорее всего это связано с довольно небольшим количеством данных и плохим датасетом.
В датасете кроме речи также есть мысли персонажей, что тоже надо убрать, но сделать это автоматически затруднительно, и это сделает датасет еще меньше. Кроме того, как можно заметить по тестовому разговору, бот научился, что можно отвечать знаками препинания, что также скорее всего является следствие особенностей датасета.